In [21]:
import yfinance as yf
import itertools
from collections import defaultdict
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

def get_exchange_rates():
    currencies = ['USD', 'EUR', 'JPY', 'GBP', 'CHF', 'CAD', 'AUD', 'NZD', 'CNY', 'INR']
    rates = {}
    for base in currencies:
        for quote in currencies:
            if base != quote:
                pair = f"{base}{quote}=X"
                try:
                    data = yf.Ticker(pair).history(period="1d")
                    if not data.empty:
                        rates[f"{base}/{quote}"] = data['Close'].iloc[-1]
                    else:
                        print(f"No data available for {pair}")
                except Exception as e:
                    print(f"Error fetching data for {pair}: {e}")
    print(rates)
    return rates

# Function to train and predict exchange rates using Random Forest
def predict_random_forest(base, quote, interval_minutes=15, num_intervals=12):
    pair = f"{base}/{quote}"
    # Fetch historical data
    data = yf.Ticker(f"{base}{quote}=X").history(period="1mo")
    if data.empty or len(data) < 50:
        return None
    
    # Prepare data
    X = np.arange(len(data)).reshape(-1, 1)
    y = data['Close'].values
    
    # Train Random Forest model
    model = RandomForestRegressor()
    model.fit(X, y)
    
    # Predict next 3 days at 15-minute intervals
    forecast = []
    for i in range(num_intervals):
        prediction = model.predict(np.array([len(data) + i]).reshape(-1, 1))
        forecast.append(prediction[0])
    
    return forecast

# Function to train and predict exchange rates using LightGBM
def predict_lightgbm(base, quote, interval_minutes=15, num_intervals=12):
    pair = f"{base}/{quote}"
    # Fetch historical data
    data = yf.Ticker(f"{base}{quote}=X").history(period="1mo")
    if data.empty or len(data) < 50:
        return None
    
    # Prepare data
    X = np.arange(len(data)).reshape(-1, 1)
    y = data['Close'].values
    
    # Train LightGBM model
    model = LGBMRegressor()
    model.fit(X, y)
    
    # Predict next 3 days at 15-minute intervals
    forecast = []
    for i in range(num_intervals):
        prediction = model.predict(np.array([len(data) + i]).reshape(-1, 1))
        forecast.append(prediction[0])
    
    return forecast

# Function to train and predict exchange rates using XGBoost
def predict_xgboost(base, quote, interval_minutes=15, num_intervals=12):
    pair = f"{base}/{quote}"
    # Fetch historical data
    data = yf.Ticker(f"{base}{quote}=X").history(period="1mo")
    if data.empty or len(data) < 50:
        return None
    
    # Prepare data
    X = np.arange(len(data)).reshape(-1, 1)
    y = data['Close'].values
    
    # Train XGBoost model
    model = XGBRegressor()
    model.fit(X, y)
    
    # Predict next 3 days at 15-minute intervals
    forecast = []
    for i in range(num_intervals):
        prediction = model.predict(np.array([len(data) + i]).reshape(-1, 1))
        forecast.append(prediction[0])
    
    return forecast

# Function to train and predict exchange rates using CatBoost
def predict_catboost(base, quote, interval_minutes=15, num_intervals=12):
    pair = f"{base}/{quote}"
    # Fetch historical data
    data = yf.Ticker(f"{base}{quote}=X").history(period="1mo")
    if data.empty or len(data) < 50:
        return None
    
    # Prepare data
    X = np.arange(len(data)).reshape(-1, 1)
    y = data['Close'].values
    
    # Train CatBoost model
    model = CatBoostRegressor(verbose=0)
    model.fit(X, y)
    
    # Predict next 3 days at 15-minute intervals
    forecast = []
    for i in range(num_intervals):
        prediction = model.predict(np.array([len(data) + i]).reshape(-1, 1))
        forecast.append(prediction[0])
    
    return forecast

# Function to calculate arbitrage given predicted rates
def calculate_arbitrage(predicted_rates):
    opportunities = []

    for length in range(3, 6):  # 3 to 5 currency combinations
        for path in itertools.permutations(currencies, length):
            path = list(path) + [path[0]]  # Complete the cycle
            profit = 1
            for i in range(len(path) - 1):
                current_pair = f"{path[i]}/{path[i+1]}"
                next_pair = f"{path[i+1]}/{path[i+2]}"
                
                if current_pair in predicted_rates and next_pair in predicted_rates:
                    current_rates = predicted_rates[current_pair]
                    next_rates = predicted_rates[next_pair]
                    if current_rates and next_rates:
                        profit *= current_rates[-1] / next_rates[0]
                    else:
                        # Handle the case where rates are empty or None
                        print(f"Empty or None rates for {current_pair} or {next_pair}. Skipping.")
                        profit = 0
                        break
                else:
                    # Handle the case where one of the pairs is missing
                    print(f"Missing rates for {current_pair} or {next_pair}. Skipping.")
                    profit = 0
                    break
            
            if profit > 0:
                opportunities.append((path, profit - 1))

    return sorted(opportunities, key=lambda x: x[1], reverse=True)

def main():
    print("Fetching exchange rates...")
    rates = get_exchange_rates()
    
    if not rates:
        print("No exchange rate data` available. Exiting.")
        return

    # Predict exchange rates for the next 3 days using different models
    predicted_rates = {}
    models = {
        'Random Forest': predict_random_forest,
        'LightGBM': predict_lightgbm,
        'CatBoost': predict_catboost,
        'XGBoost': predict_xgboost
    }

    for base in currencies:
        for quote in currencies:
            if base != quote:
                for model_name, model_func in models.items():
                    try:
                        prediction = model_func(base, quote)
                        if prediction is not None:
                            predicted_rates[f"{base}/{quote}/{model_name}"] = prediction
                    except Exception as e:
                        print(f"Error predicting {base}/{quote} using {model_name}: {e}")

    # Calculate arbitrage opportunities based on predicted rates
    opportunities = calculate_arbitrage(predicted_rates)

    if not opportunities:
        print("No profitable arbitrage opportunities found.")
    else:
        print("\nTop 10 Most Profitable Arbitrage Opportunities:")
        for i, (path, profit) in enumerate(opportunities[:10], 1):
            print(f"{i}. Path: {' -> '.join(path)}")
            print(f"   Potential profit: {profit:.4%}")

    print("\nArbitrage opportunities by number of currencies involved:")
    opportunities_by_length = defaultdict(list)
    for path, profit in opportunities:
        opportunities_by_length[len(path) - 1].append((path, profit))

    for length in range(3, 6):
        top_opportunities = sorted(opportunities_by_length[length], key=lambda x: x[1], reverse=True)[:5]
        print(f"\nTop 5 {length}-currency arbitrage opportunities:")
        for i, (path, profit) in enumerate(top_opportunities, 1):
            print(f"{i}. Path: {' -> '.join(path)}")
            print(f"   Potential profit: {profit:.4%}")

if __name__ == "__main__":
    main()

Fetching exchange rates...
{'USD/EUR': 0.9332000017166138, 'USD/JPY': 160.8350067138672, 'USD/GBP': 0.7907400131225586, 'USD/CHF': 0.8983200192451477, 'USD/CAD': 1.3676999807357788, 'USD/AUD': 1.4991999864578247, 'USD/NZD': 1.6426000595092773, 'USD/CNY': 7.26669979095459, 'USD/INR': 83.33499908447266, 'EUR/USD': 1.0715816020965576, 'EUR/JPY': 172.38299560546875, 'EUR/GBP': 0.847350001335144, 'EUR/CHF': 0.962369978427887, 'EUR/CAD': 1.4648000001907349, 'EUR/AUD': 1.6059000492095947, 'EUR/NZD': 1.7590999603271484, 'EUR/CNY': 7.7804999351501465, 'EUR/INR': 89.31900024414062, 'JPY/USD': 0.00621755188331008, 'JPY/EUR': 0.0058003999292850494, 'JPY/GBP': 0.004910999909043312, 'JPY/CHF': 0.005576999858021736, 'JPY/CAD': 0.008497999981045723, 'JPY/AUD': 0.009317999705672264, 'JPY/NZD': 0.010211000218987465, 'JPY/CNY': 0.04513999819755554, 'JPY/INR': 0.5178899765014648, 'GBP/USD': 1.2646381855010986, 'GBP/EUR': 1.1796000003814697, 'GBP/JPY': 203.43099975585938, 'GBP/CHF': 1.1358000040054321, 'GB

In [ ]:
jii